In [4]:
file_name = 'ID59_ALD Beam test'

In [1]:
import gdsfactory as gf
from itertools import chain, repeat
from blocks import doubly_clamped_beam_with_round_support, create_deep_etch_mask, stair, big_mark_set, text_outline
width_list = [0.05, 0.1, 0.5, 1, 2, 5]
length_list = [1000,500,300,100]

width_list_repeat = list(chain.from_iterable(repeat(width_list, 3)))

beam_spacing = 15


def beam_with_anchor(width, length, support_length, anchor_size:tuple):
    c = gf.Component()
    beam = c << doubly_clamped_beam_with_round_support(width, length, support_length)
    anchor_1 = c << gf.components.rectangle(size=anchor_size, port_type='placement')
    anchor_2 = c << gf.components.rectangle(size=anchor_size, port_type='placement')
    anchor_1.connect("e3", beam.ports["w1"], allow_width_mismatch=True)
    anchor_2.connect("e1", beam.ports["e1"], allow_width_mismatch=True)
    label = text_outline('abcd',size=5, mask_layer='DEEP_ETCH')
    c.add_ref(label)
    c.flatten()
    return c

2025-08-15 21:43:03.995 | INFO     | kfactory.kcell:show:3948 - klive v0.4.1: Opened file 'c:\Users\xwei2\Documents\Python Projects\NOEMS_Layout\build\gds\1759169939.oas'


In [2]:
stair_offset = 10
y=0
c = gf.Component()
for length in length_list:
    for width in width_list_repeat:
        ref = c.add_ref(beam_with_anchor(width, length, 1, (10, 10)))
        ref.movey(y)
        y += beam_spacing
    y += stair_offset
h_stair = [beam_spacing*len(width_list_repeat)+stair_offset]*len(length_list)
w_stair = [500, 200, 200, 100]
stair_trench = stair(w_stair, h_stair, layer=(2,0)).copy()
stair_trench.movex(-1000)
stair_trench.movey(-stair_offset)
trench = c << stair_trench
poly = stair_trench.get_polygons()[gf.get_layer((2, 0))][0]
sized_poly = poly.sized(15e3)
c.add_polygon(sized_poly, layer='DEEP_ETCH')
c2 = gf.Component()
beam_mask = gf.boolean(c,c,'not','DEEP_ETCH','DEEP_ETCH','WG')
y = -3
for length in length_list:
    for width in width_list_repeat:
        label = text_outline(f'{width}x{length}',size=12, mask_layer='DEEP_ETCH', outline_width=0.5, with_mask=False)
        label_ref = c2 << label
        label_ref.movex(25)
        label_ref.movey(y)
        label_ref2 = c2 << label
        label_ref2.movex(25+1500)
        label_ref2.movey(y)
        
        label_ref3 = c2 << label
        label_ref3.movex(-length-100)
        label_ref3.movey(y)

        label_ref4 = c2 << label
        label_ref4.movex(25+1500 - length - 100)
        label_ref4.movey(y)
        y += beam_spacing
    y += stair_offset
c2.add_ref(beam_mask)
beam_ref2 = c2 << beam_mask
beam_ref2.movex(1500)
c2.add_ref(stair_trench)
marker = big_mark_set()
marker_l = c2 << marker
marker_r = c2 << marker
marker_l.movex(-2500)
marker_r.movex(2500)
c2.show()

In [ ]:
c2.write_gds(fr'.\gds_files\{file_name}.gds')

WindowsPath('gds_files/ID59_ALD Beam test.gds')

: 